<a href="https://colab.research.google.com/github/ms1515/AI-NLP/blob/main/model_pretraining/Final_BERT_Pretraining_with_corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers==4.18.0 sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 5.2 MB/s 
     |████████████████████████████████| 4.0 MB 76.5 MB/s 
     |████████████████████████████████| 1.3 MB 69.9 MB/s 
     |████████████████████████████████| 6.6 MB 75.7 MB/s 
     |████████████████████████████████| 880 kB 74.8 MB/s 
     |████████████████████████████████| 120 kB 74.1 MB/s 
     |████████████████████████████████| 115 kB 79.3 MB/s 
     |████████████████████████████████| 212 kB 83.8 MB/s 
     |████████████████████████████████| 127 kB 81.3 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=77f057b83210311d8a5ea38ac0be5d11486bd3b45b8ee57f6866bd868afe1cb8
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
   

In [2]:
from datasets import *
from transformers import *
from tokenizers import *
import os
import json

In [135]:
# download and prepare security corpus dataset
import tensorflow as tf

url = 'https://github.com/eyalmazuz/ThreatIntelligenceCorpus/archive/refs/heads/master.zip' 


dataset = tf.keras.utils.get_file('master.zip', url,
                                  extract=True, cache_dir='.',
                                  cache_subdir='')

# dataset = load_dataset("text", data_dir='ThreatIntelligenceCorpus-master/Corpus', split='train')

dataset = load_dataset("text", data_files=['ThreatIntelligenceCorpus-master/Corpus/*.txt'], split='all')


Resolving data files:   0%|          | 0/20560 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-47e0d3bfe822e4c3/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad. Subsequent calls will reuse this data.


In [138]:
dataset

Dataset({
    features: ['text'],
    num_rows: 932093
})

In [139]:
# split the dataset into training (90%) and testing (10%)
d = dataset.train_test_split(test_size=0.1)
d["train"], d["test"]

(Dataset({
     features: ['text'],
     num_rows: 838883
 }), Dataset({
     features: ['text'],
     num_rows: 93210
 }))

In [140]:
for t in d["train"]["text"][:3]:
  print(t)
  print("="*50)

The satellite network is referred to as a global area network (GAN), and within that is the broadband global area network (BGAN) for satellite data. The BGAN and 5G take advantage of each other in a variety of ways that bring benefits to the IoT, specifically the industrial internet of things (IIoT). These benefits include reduced operating expenses (OPEX) and IoT management methods such as large-area update and modification of IoT systems and devices through firmware and software updates. One example of the latter is the use of a satellite to update the subscriber identity module (SIM) cards in mobile IoT devices such as autonomous vehicles. This “efficient content delivery” is planned for a variety of reasons, such as its bypassing of a degree of telecom complexity and expense by allowing 5G devices to be accessed directly by satellite.
Several U.S. defense contractors and small, private security companies also sell vulnerability details and exploits. VUPEN is the most visible and vo

In [7]:
# if you have your custom dataset 
# dataset = LineByLineTextDataset(
#     tokenizer=tokenizer,
#     file_path="path/to/data.txt",
#     block_size=64,
# )

In [8]:
# or if you have huge custom dataset separated into files
# load the splitted files
# files = ["train1.txt", "train2.txt"] # train3.txt, etc.
# dataset = load_dataset("text", data_files=files, split="train")

In [141]:
# if you want to train the tokenizer from scratch (especially if you have custom
# dataset loaded as datasets object), then run this cell to save it as files
# but if you already have your custom data as text files, there is no point using this
def dataset_to_text(dataset, output_filename="data.txt"):
  """Utility function to save dataset text to disk,
  useful for using the texts to train the tokenizer 
  (as the tokenizer accepts files)"""
  with open(output_filename, "w") as f:
    for t in dataset["text"]:
      print(t, file=f)

# save the training set to train.txt
dataset_to_text(d["train"], "train.txt")
# save the testing set to test.txt
dataset_to_text(d["test"], "test.txt")

In [142]:
special_tokens = [
  "[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "<S>", "<T>"
]
# if you want to train the tokenizer on both sets
# files = ["train.txt", "test.txt"]
# training the tokenizer on the training set
files = ["train.txt"]
# 30,522 vocab is BERT's default vocab size, feel free to tweak
vocab_size = 30_522
# maximum sequence length, lowering will result to faster training (when increasing batch size)
max_length = 1024
# whether to truncate
truncate_longer_samples = False

In [143]:
# initialize the WordPiece tokenizer
tokenizer = BertWordPieceTokenizer()
# train the tokenizer
tokenizer.train(files=files, vocab_size=vocab_size, special_tokens=special_tokens)
# enable truncation up to the maximum 512 tokens
tokenizer.enable_truncation(max_length=max_length)

In [144]:
model_path = "pretrained-security-bert"
# make the directory if not already there
if not os.path.isdir(model_path):
  os.mkdir(model_path)

In [145]:
# save the tokenizer  
tokenizer.save_model(model_path)

['pretrained-security-bert/vocab.txt']

In [146]:
# dumping some of the tokenizer config to config file, 
# including special tokens, whether to lower case and the maximum sequence length
with open(os.path.join(model_path, "config.json"), "w") as f:
  tokenizer_cfg = {
      "do_lower_case": True,
      "unk_token": "[UNK]",
      "sep_token": "[SEP]",
      "pad_token": "[PAD]",
      "cls_token": "[CLS]",
      "mask_token": "[MASK]",
      "model_max_length": max_length,
      "max_len": max_length,
  }
  json.dump(tokenizer_cfg, f)

In [147]:
# when the tokenizer is trained and configured, load it as BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained(model_path)

Didn't find file pretrained-security-bert/tokenizer.json. We won't load it.
Didn't find file pretrained-security-bert/added_tokens.json. We won't load it.
Didn't find file pretrained-security-bert/special_tokens_map.json. We won't load it.
Didn't find file pretrained-security-bert/tokenizer_config.json. We won't load it.
loading file pretrained-security-bert/vocab.txt
loading file None
loading file None
loading file None
loading file None
loading configuration file pretrained-security-bert/config.json
Model config BertConfig {
  "_name_or_path": "pretrained-security-bert",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "cls_token": "[CLS]",
  "do_lower_case": true,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "mask_token": "[MASK]",
  "max_len": 1024,
  "max_position_embeddings": 512,
  "model_max_length": 1024,
  "model_type": "bert",
  "num_att

In [148]:
def encode_with_truncation(examples):
  """Mapping function to tokenize the sentences passed with truncation"""
  return tokenizer(examples["text"], truncation=True, padding="max_length",
                   max_length=max_length, return_special_tokens_mask=True)

def encode_without_truncation(examples):
  """Mapping function to tokenize the sentences passed without truncation"""
  return tokenizer(examples["text"], return_special_tokens_mask=True)

# the encode function will depend on the truncate_longer_samples variable
encode = encode_with_truncation if truncate_longer_samples else encode_without_truncation

# tokenizing the train dataset
train_dataset = d["train"].map(encode, batched=True)
# tokenizing the testing dataset
test_dataset = d["test"].map(encode, batched=True)

if truncate_longer_samples:
  # remove other columns and set input_ids and attention_mask as PyTorch tensors
  train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
  test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
else:
  # remove other columns, and remain them as Python lists
  test_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])
  train_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])

  0%|          | 0/839 [00:00<?, ?ba/s]

  0%|          | 0/94 [00:00<?, ?ba/s]

In [149]:
from itertools import chain
# Main data processing function that will concatenate all texts from our dataset and generate chunks of
# max_seq_length.
# grabbed from: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= max_length:
        total_length = (total_length // max_length) * max_length
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + max_length] for i in range(0, total_length, max_length)]
        for k, t in concatenated_examples.items()
    }
    return result

# Note that with `batched=True`, this map processes 1,000 texts together, so group_texts throws away a
# remainder for each of those groups of 1,000 texts. You can adjust that batch_size here but a higher value
# might be slower to preprocess.
#
# To speed up this part, we use multiprocessing. See the documentation of the map method for more information:
# https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map
if not truncate_longer_samples:
  train_dataset = train_dataset.map(group_texts, batched=True,
                                    desc=f"Grouping texts in chunks of {max_length}")
  test_dataset = test_dataset.map(group_texts, batched=True,
                                  desc=f"Grouping texts in chunks of {max_length}")
  # convert them from lists to torch tensors
  train_dataset.set_format("torch")
  test_dataset.set_format("torch")

Grouping texts in chunks of 1024:   0%|          | 0/839 [00:00<?, ?ba/s]

Grouping texts in chunks of 1024:   0%|          | 0/94 [00:00<?, ?ba/s]

In [150]:
len(train_dataset), len(test_dataset)

(16070, 1823)

In [151]:
# initialize the model with the config
model_config = BertConfig(vocab_size=vocab_size, max_position_embeddings=max_length)
model = BertForMaskedLM(config=model_config)

In [152]:
# initialize the data collator, randomly masking 20% (default is 15%) of the tokens for the Masked Language
# Modeling (MLM) task
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [153]:
training_args = TrainingArguments(
    output_dir=model_path,          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # evaluate each `logging_steps` steps
    overwrite_output_dir=True,      
    num_train_epochs=10,            # number of training epochs, feel free to tweak
    per_device_train_batch_size=10, # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=8,  # accumulating the gradients before updating the weights
    per_device_eval_batch_size=64,  # evaluation batch size
    logging_steps=1000,             # evaluate, log and save model checkpoints every 1000 step
    save_steps=1000,
    # load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    # save_total_limit=3,           # whether you don't have much space so you let only 3 model weights saved in the disk
)

using `logging_steps` to initialize `eval_steps` to 1000
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [154]:
# initialize the trainer and pass everything to it
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [155]:
import torch
torch.cuda.empty_cache()

In [ ]:
# train the model
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 16070
  Num Epochs = 10
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 80
  Gradient Accumulation steps = 8
  Total optimization steps = 2000


Step,Training Loss,Validation Loss


In [ ]:
model.save_pretrained(model_path)

In [ ]:
# when you load from pretrained

saved_model = BertForMaskedLM.from_pretrained(model_path)
tokenizer = BertTokenizerFast.from_pretrained(model_path)
# or simply use pipeline
fill_mask = pipeline("fill-mask", model=saved_model, tokenizer=tokenizer)

In [ ]:
# perform predictions
example = "It is known that cyber [MASK] are dangerous"
for prediction in fill_mask(example):
  print(prediction)

In [ ]:
# perform predictions
examples = [
  "Today's most trending cyber [MASK] is SQL Injection",
  "The [MASK] attack was yesterday, but today it's safe.",
]
for example in examples:
  for prediction in fill_mask(example):
    print(f"{prediction['sequence']}, confidence: {prediction['score']}")
  print("="*50)

# Fine tuning on report dataset

In [ ]:
url = 'https://github.com/eyalmazuz/AttackAttributionDataset/archive/refs/heads/master.zip' 

dataset = tf.keras.utils.get_file('master(1).zip', url,
                                  extract=True, cache_dir='.',
                                  cache_subdir='')


In [ ]:
import tensorflow_hub as hub

from transformers import TFAutoModelForSequenceClassification


model = TFAutoModelForSequenceClassification.from_pretrained(model_path, num_labels=12, from_pt=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
report_dataset = load_dataset("text", data_files=['AttackAttributionDataset-master/**/*.txt', ], split="all", sample_by="document")
report_dataset

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


# split the dataset into training (90%) and testing (10%)
report_d = report_dataset.train_test_split(test_size=0.1)

tokenized_train_ds = report_d["train"].map(tokenize_function, batched=True)
tokenized_test_ds = report_d["test"].map(tokenize_function, batched=True)

small_train_dataset = tokenized_train_ds.shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_test_ds.shuffle(seed=42).select(range(1000))


tf_train_dataset = tokenized_train_ds.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=5,
)

tf_validation_dataset = tokenized_test_ds.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=5,
)

In [ ]:
## Native TF dataset from file directory
# batch_size = 5
# seed = 42

# train_ds = tf.keras.utils.text_dataset_from_directory(
#     'AttackAttributionDataset-master',
#     batch_size=batch_size,
#     validation_split=0.15,
#     subset='training',
#     seed=seed)


# val_ds = tf.keras.utils.text_dataset_from_directory(
#     'AttackAttributionDataset-master', batch_size=batch_size, validation_split=0.15,
#     subset='validation', seed=seed)

In [ ]:



model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(x=train_ds, validation_data=val_ds, epochs=15)

In [ ]:
!nvidia-smi